# Branch and Bound
## Un tutorial del curso MA4702. Universidad de Chile. 2020
### Profesor José Soto


Uno de los métodos más usados para resolver PLM y otros problemas de optimización es *branch and bound (BnB)*. Se basa en la siguiente idea: 


>  **IDEA** 
>
>Sea $\{S_1,\dots, S_k\}$ una partición del conjunto factible $S$ de un problema de optimización $z^*=\max \{c^Tx\colon x\in S\}$.
>
>Si $z^*_i=\max\{c^Tx\colon x\in S_i\}$ entonces $z^*=\max_{i\in [k]}z^*_i.$
    

BnB consiste en particionar el conjunto factible $S$ del problema original en conjuntos más pequeños y resolver luego $\max c^Tx$ en cada subconjunto de manera recursiva. Si la recursión se pudiera llevar completamente, al final  enumeraríamos todos los puntos factibles del problema. Esta idea tiene dos problemas. Primero, si el dominio es infinito entonces esto no es posible. Segundo, incluso si el dominio es finito, esto podría ser extremadamente lento y no difiere en nada de simplemente de simple fuerza bruta. 
    La idea es que BnB no explore todo el árbol de recursión sino que guarde cotas para los subproblemas que ya ha resuelto y, usando estas cotas determinar que no necesitamos resolver ciertos subproblemas. 

 En lo que sigue nos enfocaremos en PLM de la forma    
 $\begin{aligned}
    \qquad (M)&\quad \max c^Tx\\
    S&\colon \begin{cases}
    Ax &\leq b\\
    x&\in \mathbb{Z}^E\times \mathbb{R}^C
    \end{cases}
    \end{aligned}$
    
 donde $A$, $b$, $c$ son racionales.

Llamemos $P=\{x\in \mathbb{R}^{E+ C}, Ax\leq b\}$ a la relajación lineal natural de $S$. Tenemos $S \subseteq P.$  Como los datos son racionales tenemos que el programa lineal relajado

 $\begin{aligned}
    \qquad (L)&\quad \max c^Tx\\
    P&\colon \begin{cases}
    Ax &\leq b\\
    x&\in \mathbb{R}^{E\cup C}
    \end{cases}
    \end{aligned}$
    
    
es o bien **infactible**, o bien **factible no acotado** o bien **factible acotado** con solución óptima $\bar{x}\in P$ racional. En este caso llamamos $\bar{z}=c^T\bar{x}$ a su valor.
    

Llamemos $(M_0)$ al problema inicial dado y **pediremos que $(P_0)$ sea factible acotado**. (Lo pedimos pues  BnB *a secas* no es capaz de lidiar con problemas con relajación no acotada en la dirección de optimización)

Para resolver $(M_0)$ se construye **iterativamente** un árbol $\mathcal{T}$cuyos nodos son de la forma $(M,B,s)$ con $(M)$ un subproblema de $(M_0)$, $B$ una cota superior (optimista) del valor de $(M_0)$, y $s$ un **status** que puede ser **activo**, **ramificado**, **infactible**, **dominado** o **entero**. Tanto la cota como el status de un nodo puede cambiar a lo largo del algoritmo.

En el árbol  siempre se tendrá que **la unión de los dominios de las hojas** es el **dominio** de la raíz. Además se mantiene globalmente una solución factible (inicialmente nula) $x^*$ llamada **incumbente** que resulta ser la mejor solución encontrada hasta el momento y $z^*=c^Tx^*$ su valor (también llamada **mejor cota inferior**).
    Inicialmente, el nodo raíz es $(M_0,B_0,\text{activo})$ donde $M_0$ es el PLM original y $B_0$ es el valor $\bar{z}$ de su relajación $L_0$. Además, $x^*$ es nulo y $z^*=-\infty$. Durante el algoritmo BnB hay 2 procesos importantes: la **creación de un nodo** y la **ramificación de un nodo activo**.

## Creación de un nodo

Algoritmo: Creación de un nodo $(M)$
1. Resolver la relajación lineal $(P)$ de $(M)$ 
2. Si $(P)$ es infactible, **return** $(M,-\infty, \text{infactible})$
3. Si el óptimo $\bar{x}$ (con valor $\bar{z}=c^T\bar{x}$) de $(P)$  es factible en $(M)$ entonces
    1. Si $\bar{z}\leq \bar{z}^*$, **return** $(M,\bar{z}, \text{entero})$
    2. Si $\bar{z}> \bar{z}^*$, entonces $x^*\gets \bar{x}, z^*\gets \bar{z}$, **return** $(M,\bar{z},\text{entero})$ // actualizando incumbente
4. Si el óptimo $\bar{x}$ (con valor $\bar{z}=c^T\bar{x}$) de $(P)$  es infactible en $(M)$ entonces
    1. Si $\bar{z}\leq \bar{z}^*$, **return** $(M,\bar{z}, \text{dominado})$
    2. Si $\bar{z}> \bar{z}^*$, **return** $(M,\bar{z}, \text{activo})$

  
    Notamos que si un nodo se declara entero, entonces conocemos su mejor valor factible. Si un nodo se declara dominado en su dominio $S$ no pueden haber soluciones enteras mejores que el incumbente, por lo que no es necesario seguir procesándolo, al igual que si el nodo se declara infactible. Los únicos problemas que podrían tener soluciones enteras mejores que el incumbente actual son aquellos que están activos.

## Ramificación de un nodo activo

Algoritmo: Ramificación de un nodo $(M, B(M), \text{activo})$
1. Determinar $k\geq 2$ subproblemas $(M_i)$ cuyos dominios particionen el dominio de $(M)$
2. Crear un nodo $(M_i)$ para cada subproblema y colgarlo como hijo de $(M)$
3. Declarar el status de $M$ como ramificado.

    Hay varias formas de ramificar un nodo. Una forma estándar y simple es hacer **branching en una variable**
    
   ### Branching en una variable $x_k$ en el nodo $(M)$
    
   Como el óptimo $\bar{x}\in P$ de $(L)$ no es factible en $(M)$ debe haber una coordenada $k\in E$ tal que la variable $\bar{x}_k$ es fraccional (pero que debería ser entera en $(M)$). Podemos **elegir** una coordenada y definir entonces dos PLM nuevos $(M_1)$ y $(M_2)$ como sigue:
    \begin{align*}
    S_1&=S\cap \{x\colon x_k \leq \lfloor \bar{x}_k \rfloor\}. &S_2&=S\cap \{x\colon x_k \geq \lceil \bar{x}_k \rceil\}.\\
    P_1&= P_0 \cap \{x\colon x_k \leq \lfloor \bar{x}_k \rfloor\}. &P_2&= P_0 \cap \{x\colon x_k \geq \lceil \bar{x}_k \rceil\}.\\
    (M_1)&\colon \max\{c^Tx\colon x\in S_1\}. & (M_2)&\colon \max\{c^Tx\colon x\in S_2\}.\\
    (L_1)&\colon \max\{c^Tx\colon x\in P_1\}. & (L_2)&\colon \max\{c^Tx\colon x\in P_2\}.
    \end{align*}
    

Ahora estamos listos para escribir el algoritmo de BnB completo. Como BnB es un método genérico hay algunas instrucciones que no están completamente descritas. 
    
    
Algoritmo: Branch and Bound

Entrada: Un PLM $(M_0)$ **racional** con relajación $(L_0)$ factible acotada.
1. $x^*\gets *null*$, crear nodo $(M_0)$ como raíz del árbol $\mathcal{T}$.
2. Mientras existan nodos activos en $\mathcal{T}$
    1. Si se ha alcanzado un criterio de terminación temprana *PARAR*.
    2. *ELEGIR* un nodo activo $(M,B,\text{activo})$ y ramificarlo.
    3. Actualizar las cotas $B(M')$ para cada nodo $(M')$ en el camino entre $(M)$ y la raíz $(M_0)$, i.e.:
        1. $B(M')\gets \min\{B(M'), \max\{B(\tilde{M})\colon (\tilde{M}) \text{ es hijo de } (M')\}\}\}$
    4. Si el incumbente cambió, actualizar todos los nodos activos que ahora estén dominados, i.e.:
        1. Declarar todo $(M',B',\text{activo})$ con $B'\leq z^*$ como dominado.
3. **Return** $x^*$.
    	

> **Observación importante**: Los dominios de los PLM asociados a nodos ACTIVOS **particionan** el dominio del PLM original en todo momento

Discutiremos más adelante posibles criterios de terminación temprana. Pero si en algún minuto necesitamos terminar, entonces observamos que el valor óptimo de $(M_0)$ siempre está en $[z^*, B(M_0)]$. 

La razón $\frac{B(M_0)-z^*}{z^*}$ se suele llamar el **GAP de resolución** (en dicho momento). Hagamos un ejemplo concreto de BnB usando solo branchings por variables. Luego de ese ejemplo daremos varias observaciones importantes.







# EJEMPLO

(El árbol de branch and bound está disponible en ucursos)

 \begin{align*}
    M_0&:\quad 	\max 3x+5y\\
    S_0&:\quad \begin{cases}
    20y+9x&\leq 74\\
    25y+18x&\leq 105\\
    x,y&\geq 0\\
    x,y&\in \mathbb{Z}
    \end{cases}
    \end{align*}

Dibujemos el área factible para entender el PL un poco más antes de partir.

In [ ]:
#using Pkg
#Pkg.add("Plots")
#Pkg.add("Plotly")
using Plots
plotly()
f(x) = (74-9x)/20
g(x) = (105-18x)/25
X = [0,6]

plot(X, [f,g], fill = (0, 0.5, :auto), leg=false)
plot!([0, 1], [0, 5/3], lw=3)

annotate!(2, 2, "S0", :color, arrow=3)

El área café es el área factible $P_0$ de $(M_0)$ y buscamos la mejor solución entera.  Usemos Gurobi para resolver la relajación del problema $(M_0)$. Para esto ejecutemos el siguiente código.

In [ ]:
using JuMP, Gurobi

In [ ]:
#Creamos el objeto modelo
modelo= Model()

#Le indicamos a JuMP que el solver a utilizar es Gurobi y eliminamos presolver
set_optimizer(modelo, Gurobi.Optimizer)
set_optimizer_attributes(modelo, "Presolve" => 0, "OutputFlag" => 0) 

#declaración de variables de decisión
@variable(modelo, x>=0)
@variable(modelo, y>=0)

#restricciones
@constraint(modelo,rest1, 20y + 9x<=74)
@constraint(modelo, rest2, 25y+ 18x<=105)

#función objetivo
@objective(modelo, Max, 3x+5y)

In [ ]:
#resolver (CUIDADO, tenemos outputflag 0)

optimize!(modelo)
termination_status(modelo)

In [ ]:
#Solucion encontrada: imprimir
x0=value(modelo[:x])
y0=value(modelo[:y])
z0=objective_value(modelo)

println("z0=",z0," x0=", x0," y0=", y0)

 Al resolver la relajación lineal anterior, encontramos que el óptimo del PL asociado es $$p_0=(x_0,y_0)\approx (1.8518, 2.866), z_0\approx 19.888.$$

In [ ]:
#Agregar el punto encontrado al grafico
scatter!([x0],[y0])

Como el punto no es entero al crear el nodo $(M_0)$ en el árbol, lo debemos marcar como activo en el árbol BnB.

Como ambas coordenadas de $(x_0,y_0)$ son fraccionales podemos elegir una, digamos $x$ y **ramificar** de acuerdo a dicha variable, creando dos problemas $M_1$ y $M_2$, con conjuntos factibles 
$$S_1=S_0\cap \{x\leq \lfloor x_0\rfloor\}$$

y

$$S_1=S_0\cap \{x\geq \lceil x_0\rceil\}.$$ 
    


 \begin{align*}
 M_1&:\quad 	\max 3x+5y & M_2&:\quad 	\max 3x+5y\\ 
 S_1&:\quad \begin{cases}
 20y+9x&\leq 74\\
 25y+18x&\leq 105\\
 x&\leq 1\\
 x,y&\geq 0\\
 x,y&\in \mathbb{Z}\end{cases}&S_2&:\quad \begin{cases}
 20y+9x&\leq 74\\
 25y+18x&\leq 105\\
 x&\geq 2\\
 x,y&\geq 0\\
 x,y&\in \mathbb{Z}\end{cases}
 \end{align*}

Creemos entonces los nodos asociados a $(M_1)$ y $(M_2)$, resolviendo sus programas lineales y viendo en que caso nos encontramos

In [ ]:
#Primero resolvemos M1. M1 es igual a M0 solo que agregamos la cota x<=1
set_upper_bound(modelo[:x], 1) 

#mirar modelo
modelo

In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

In [ ]:
x1=value(modelo[:x])
y1=value(modelo[:y])
z1=objective_value(modelo)

println("z1=",z1," x1=", x1," y1=", y1)

In [ ]:
#Ahora resolvamos el modelo 2. Debemos cambiar 0<=x<=1 por x>=2
delete_upper_bound(modelo[:x]) 
set_lower_bound(modelo[:x], 2)

#mirar modelo
modelo


In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)


In [ ]:
x2=value(modelo[:x])
y2=value(modelo[:y])
z2=objective_value(modelo)

println("z2=",z2," x2=", x2," y2=", y2)

  La solución óptima de la relajación de  $M_1$ es $(x_1,y_1)=(1;3.25)$ de valor $z_1=19.25$, y la solución óptima de la relajación de $M_2$ es $(x_2,y_2)=(2;2.76)$ de valor $z_2=19.8$. Luego ambos nodos se crean activos. Más aún la cota de $M_0$ se actualiza a $\max\{19.25,19.8\}=19.8$.

In [ ]:
#Rehacemos el plot anterior
f(x) = (74-9x)/20
g(x) = (105-18x)/25
X = [0,6]
plot(X, [f,g], fill = (0, 0.5, :auto), leg=false)

#zona x<1
plot!([0, 0, 1, 1], [0, 4, 4, 0], fill=(0, 0.5, :auto))
annotate!(0.5, 2, "S1", :color)

#zona x>2
plot!([2, 2, 6, 6], [0, 4, 4, 0], fill=(0, 0.5, :auto))
annotate!(3, 1, "S2", :color)

#puntos encontrados
scatter!([x0, x1, x2],[y0,y1,y2])


El area S1 es el dominio de M1 y el triangulo S2 es el dominio de M2

 Resulta útil anotar en las aristas del árbol que restricciones hemos agregado en cada problema. Ahora hay 2 nodos activos en el árbol y podemos elegir cualquiera para ramificar. La solución $(x_1,y_1)$ tiene variable $y$ fraccional. Ramifiquemos $M_1$ en dos problemas $M_3$ y $M_4$ de acuerdo a si $y\leq 3$ o si $y\geq 4$. Obtenemos:

In [ ]:
#Modelo M3
set_lower_bound(modelo[:x],0)
set_upper_bound(modelo[:x],1) 
set_upper_bound(modelo[:y],3) 

#mirar modelo
modelo

In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

¡Ojo! La solución de este PL es entera (luego el nodo asociado a M3 se declarará entero). Además, este es nuestro primer incumbente finito.

In [ ]:
x3=value(modelo[:x])
y3=value(modelo[:y])
z3=objective_value(modelo)

println("z3=",z3," x3=", x3," y3=", y3)

In [ ]:
#Ahora resolvemos M4
delete_upper_bound(modelo[:y])
set_lower_bound(modelo[:y],4) 

#mirar modelo
modelo

In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

¡El modelo M4 es infactible!

In [ ]:
#Hagamos el grafico de nuevo
f(x) = (74-9x)/20
g(x) = (105-18x)/25
X = [0,6]
plot(X, [f,g], fill = (0, 0.5, :auto), leg=false)

#zona x<=1,y<=3
plot!([0, 0, 1, 1], [0, 3, 3, 0], fill=(0, 0.5, :auto))
annotate!(0.5, 1.5, "S3", :color)

#zona x>=2
plot!([2, 2, 6, 6], [0, 4, 4, 0], fill=(0, 0.5, :auto))
annotate!(3, 1, "S2", :color)

#puntos encontrados
scatter!([x0, x1, x2, x3],[y0,y1,y2, y3])


La solución de la relajación de $M_3$ es entera $(x_3,y_3)=(1,2)$ con $z_3=18$. Por lo que $M_3$ se declara entero y además, $(x_3,y_3)$ se transforma en incumbente (actualizando también $z^*$). 

Mientras tanto la relajación de $M_4$ es infactible, por lo que su cota es $-\infty$. Actualizamos la cota superior de $M_1$ a $18$, mientras que la cota de $M_0$ se mantiene.



Como tenemos incumbente y cota, el **gap** de nuestra solución actual  es $\frac{19.8-18}{18}=\frac{1.8}{18}=0.1=10\%$. Solo queda $M_2$ activo, lo ramificamos en $M_5$ y $M_6$, donde $y\leq 2$ o $y\geq3$ respectivamente. 

In [ ]:
#Modelo M5
set_lower_bound(modelo[:x],2)
delete_upper_bound(modelo[:x])
set_upper_bound(modelo[:y],2) 
set_lower_bound(modelo[:y],0)
modelo


In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

In [ ]:
x5=value(modelo[:x])
y5=value(modelo[:y])
z5=objective_value(modelo)

println("z5=",z5," x5=", x5," y5=", y5)

In [ ]:
#Modelo M6
delete_upper_bound(modelo[:y])
set_lower_bound(modelo[:y],3) 

modelo

In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

M6 infactible.

In [ ]:
#Hagamos el grafico de nuevo
f(x) = (74-9x)/20
g(x) = (105-18x)/25
X = [0,6]
plot(X, [f,g], fill = (0, 0.5, :auto), leg=false)


#zona x<=1,y<=3
plot!([0, 0, 1, 1], [0, 3, 3, 0], fill=(0, 0.5, :auto))
annotate!(0.5, 1.5, "S3", :color)

#zona x>=2, y<=2
plot!([2, 2, 6, 6], [0, 2, 2, 0], fill=(0, 0.5, :auto))
annotate!(3, 1, "S5", :color)

#zona y<=3
plot!([0, 0, 1, 1], [0, 3, 3, 0], fill=(0, 0.5, :auto))
annotate!(0.5, 1.5, "S3", :color)


#puntos encontrados
scatter!([x0, x1, x2, x3,x5],[y0,y1,y2, y3,y5])


 Notamos que $M_6$ es infactible, y la relajación de $M_5$ tiene solución $(3.0555;2)$ con valor $z_5=19.1667$ por lo que queda activo (y podemos actualizar las cotas del árbol de BnB).

  Nuestro gap mejoró a $(19.1667-18)/18=6.48\%$. Nuevamente tenemos solo un nodo activo, $M_5$. Lo ramificamos en $x$, definiendo los problemas $M_7$ y $M_8$ con $x\leq 3$ y $x\geq 4$ respectivamente

In [ ]:
#Modelo M7
set_lower_bound(modelo[:x],0)
set_upper_bound(modelo[:x],3)
set_upper_bound(modelo[:y],2) 
set_lower_bound(modelo[:y],0)
modelo


In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

In [ ]:
x7=value(modelo[:x])
y7=value(modelo[:y])
z7=objective_value(modelo)

println("z7=",z7," x7=", x7," y7=", y7)

¡Punto integral! (nuevo incumbente)

In [ ]:
#Modelo M8
delete_upper_bound(modelo[:x])
set_lower_bound(modelo[:x],4)
modelo


In [ ]:
#resolver
optimize!(modelo)
termination_status(modelo)

In [ ]:
x8=value(modelo[:x])
y8=value(modelo[:y])
z8=objective_value(modelo)

println("z8=",z8," x8=", x8," y8=", y8)

In [ ]:
#Hagamos el grafico de nuevo
f(x) = (74-9x)/20
g(x) = (105-18x)/25
X = [0,6]
plot(X, [f,g], fill = (0, 0.5, :auto), leg=false)


#zona x<=1,y<=3
plot!([0, 0, 1, 1], [0, 3, 3, 0], fill=(0, 0.5, :auto))
annotate!(0.5, 1.5, "S3", :color)

#zona 2<=x<=3, y<=2
plot!([2, 2, 3, 3], [0, 2, 2, 0], fill=(0, 0.5, :auto))
annotate!(2.5, 1, "S7", :color)

#zona 4<=x, y<=2
plot!([4, 4, 6, 6], [0, 2, 2, 0], fill=(0, 0.5, :auto))
annotate!(4.5, 0.5, "S8", :color)


#puntos encontrados
scatter!([x0, x1, x2, x3,x5, x7, x8],[y0,y1,y2, y3,y5, y7, y8])


 Al resolver la relajación de $M_7$ se obtiene una solución **integral** $(x_7,y_7)=(3,2)$ con $z_7=19$. Como 19 es mayor que nuestro valor incumbente actual, éste se actualiza. Por otro lado al resolver la relajación de $M_8$ se obtiene una solución fraccional $(x_8,y_8)=(4;1.32)$ con $z_8=18.6$. Pero esta vez $z_8$ es peor que el valor incumbente (19), por lo que se declara **dominado**  (o podado por cota).
    
    
    Con esto hemos completado el árbol de BnB y obtenemos que la solución óptima es $(3,2)$ de valor 19.

#  Usando un solver

 Hay muchos solvers comerciales que realizan BnB de manera muy eficiente (eligen buenos nodos activos para ramificar, hacen branching en buenas variables, etc.) y de hecho aplican varias heurísticas y otros algoritmos que aceleran aún más su ejecución. Por completitud, abajo anotamos una serie de instrucciones para ejecutar GUROBI sobre nuestro modelo (y dando directrices de modo que el orden de ramificado, etc. sea el mismo que nosotros usamos en el ejemplo anterior):

In [ ]:
#Recreamos el objeto modelo
modelonuevo= Model()
@variable(modelonuevo, x>=0)
@variable(modelonuevo, y>=0)
@constraint(modelonuevo, rest1, 20y + 9x<=74)
@constraint(modelonuevo, rest2, 25y+ 18x<=105)

#función objetivo
@objective(modelonuevo, Max, 3x+5y)

#declaramos las variables como enteras
set_integer(modelonuevo[:x])
set_integer(modelonuevo[:y])

#Le indicamos a JuMP que el solver a utilizar es Gurobi y eliminamos presolver
set_optimizer(modelonuevo, Gurobi.Optimizer)
set_optimizer_attributes(modelonuevo, "Presolve" => 0, "Heuristics"=> 0, "Cuts"=> 0, "Threads" => 1, "LogToConsole" => 1, "BranchDir" => -1) 

#declaramos atributos (branch primero en x luego en y)
MOI.set(modelonuevo, Gurobi.VariableAttribute("BranchPriority"), x, 20)  
MOI.set(modelonuevo, Gurobi.VariableAttribute("BranchPriority"), y, 1)  


#resolver

optimize!(modelonuevo)
termination_status(modelonuevo)

Gurobi realiza un árbol de BnB parecido al nuestro, pero es más eficiente en su implementación: cada vez que un nodo se ramifica en 2 y uno de ellos es infactible o dominado, continúa bajando por el árbol (es decir no crea nuestros nodos 1, 4, 2, 6, 8) hasta que deja un par de nodos activos.
Otros solvers realizan otras variantes de BnB. 

# Algunas consideraciones para hacer BnB eficiente

1. **PLs similares**

En general el PL $(L)$ asociado a un nodo es muy similar al PL asociado al de su padre (son solo cotas de variable extras) por lo que podemos resolver $(L)$ de manera más eficiente a partir de la solución óptima del padre usando una iteración de **simplex-dual**.

2. **Selección de nodos activos**
    
El rendimiento de BnB depende fuertemente de la manera como se elige el nodo activo para ramificar (cuando hay varios de estos). Aquí hay dos objetivos que compiten: (1) Encontrar rápidamente un incumbente (2) Acotar rápidamente la cota $B_0$ del nodo raíz.

Algunas estrategias estándar para seleccionar nodos son 
   **Búsqueda en profundidad (DFS)**
   
   *Ventajas*: apunta a encontrar un incumbente rápidamente con pocos nodos. 
        
   *Desventajas*: puede explorar un área \emph{mala} del árbol con nodos sin buenas soluciones.
   
   **Desarrollar el mejor nodo (best-node)** Consiste en buscar el nodo activo $(M)$ cuya cota $(B)$ es la más alta posible. 
   
   *Ventajas*: Las mejores soluciones integrales se deben encontrar en nodos con cotas altas, por lo que esta estrategia apunta a encontrar rápidamente buenas soluciones (o acotar rápidamente $B_0$)
   
   *Desventajas*: Muchos nodos deben permanecer activos por largo tiempo, provocando que se deba usar una gran cantidad de memoria.
   
   **Estrategias más avanzadas** Involucran crear un estimador de cuanto debería degradarse el valor de la cota $B$ en un nodo dado de acuerdo a su PL y luego elegir aquel con mayor valor estimado para la cota. En la práctica se ocupan estrategias mixtas como por ejemplo hacer DFS hasta que se encuentre un incumbente y luego seleccionar mejor nodo. 
  
  Gurobi hace esto automáticamente (Cplex permite un poco más de control)

3. **Selección de variable a ramificar en un nodo**  
  Ver por ejemplo [Parámetro VarBranch de Gurobi](https://www.gurobi.com/documentation/9.0/refman/varbranch.html#parameter:VarBranch)
        
Puede que la solución fraccional $\bar{x}$ del nodo que ha sido elegido tenga múltiples variables fraccionales. Una forma estándar es elegir la variable $x_j$ con $j\in E$ más fraccional (la más cercana a 0.5).

Otra alternativa llamada *strong branching* involucra resolver pequeñas variaciones del PL original para determinar cual es la variable $x_j$ cuya ramificación produce el mayor decrecimiento en la cota superior $B$ del nodo a ramificar. Esta solución es 
más cara (involucra resolver un número de PL proporcional a las variables fraccionales) pero en la práctica resulta ser útil.

4. **Criterios de término**

    BnB puede tomar un tiempo prohibitivo pero podemos detener el proceso en cualquier momento y, si para entonces tenemos un incumbente, podemos retornar una solución factible y una estimación (GAP relativo) de cuan cerca de ser óptima es la solución. Criterios típicos de término temprano incluyen detener la ejecución si:
    El tiempo de reloj excede un máximo establecido, si el número de nodos procesados excede un máximo, si la memoria necesaria excede un umbral, si el gap relativo o si el gap absoluto ($B - z^*$) es menor que una tolerancia preestablecida.
    Ver por ejemplo los siguientes atributos en Gurobi

| Atributo | Descripción 
| --- | --- | 
| BarIterLimit | Limite de iteraciones para método barrera |
| BestBdStop | Parar si la mejor cota encontrada es mejor (menor) que el parámetro |
| BestObjStop | Parar si el incumbente es mejor (mayor) que el parámetro |
| Cutoff	| Indica que no estamos interesados en soluciones peores que el parámetro |
| IterationLimit | Limite de iteraciones de simplex |
| NodeLimit | Limite de nodos en el BnB |
| SolutionLimit | Limite de soluciones enteras encontradas |
| TimeLimit | Limite de tiempo total |



   
    
5. **Heurísticas**

    En muchos casos es posible determinar buenas soluciones factibles (incumbentes) en un nodo cualquiera mediante heurísticas. Por ejemplo, podemos aplicar un algoritmo simple (como glotón o programación dinámica) para encontrar una buena solución factible antes de comenzar BnB. De este modo muchas ramas pueden ser podadas rápidamente por cota al estar dominadas. Otras heurísticas típicas pueden ser usadas en cada nodo. Por ejemplo, redondear (de alguna forma inteligente) una buena solución fraccional puede producir una buena solución factible o bien, usar las soluciones enteras que podrían aparecer mientras se ejecuta SIMPLEX en un nodo. Esto se puede incorporar fácilmente al algoritmo genérico de BnB que ya vimos.
    Gurobi tiene heurísticas muy buenas incorporadas por defecto.
    Se pueden activar / desactivar cambiando el [Parámetro Heuristics de Gurobi](https://www.gurobi.com/documentation/9.0/refman/heuristics.html#parameter:Heuristics)
    
    
6. **Usar presolver**

    El presolver típicamente es capaz de reducir la complejidad del modelo de manera automática (eliminando variables, restricciones, cotas innecesarias, etc.). El presolver de Gurobi está activado por defecto.
    
    
    
7. **Usar Cortes** (branch and cut)

    En realidad BnB no es un buen método por si solo. Lo ideal es mejorar la formulación en cada nodo. Esto se puede hacer automáticamente mediante el uso de planos cortantes, como lo veremos más adelante.

    